In [1]:
from src.sentinel_api import get_sentinel_api
from src.constant import GEOJSON_PATH, DOWNLOADED_DATA_PATH, REMOVE_ZIPS
import os

# get api 
api = get_sentinel_api()

In [2]:
# search by polygon, time, and Hub query keywords
products_query = api.query(
                    limit = 1,
                    filename = 'S3A_*',
                    producttype = 'SL_2_LST___')

os.makedirs(DOWNLOADED_DATA_PATH, exist_ok = True) 
products = api.download_all(products_query, DOWNLOADED_DATA_PATH)
print(products)

Downloading: 100%|██████████| 57.2M/57.2M [00:05<00:00, 11.3MB/s]
MD5 checksumming: 100%|██████████| 57.2M/57.2M [00:01<00:00, 37.8MB/s](OrderedDict([('2c23c8da-82ff-4179-9b9d-4e45537d6319', {'id': '2c23c8da-82ff-4179-9b9d-4e45537d6319', 'title': 'S3A_SL_2_LST____20200531T153925_20200531T154225_20200531T165411_0180_059_025_4680_LN2_O_NR_004', 'size': 57227299, 'md5': '5F8AFE60571B9EE3825848D12E2A6025', 'date': datetime.datetime(2020, 5, 31, 15, 39, 25, 29000), 'footprint': 'POLYGON((136.878 -64.254,136.123 -64.5734,135.314 -64.8762,134.501 -65.1789,133.691 -65.4816,132.85 -65.7777,131.983 -66.069,131.116 -66.357,130.198 -66.6408,129.271 -66.9117,128.336 -67.1827,127.371 -67.4532,126.379 -67.7146,125.367 -67.9664,124.335 -68.2082,123.277 -68.4523,122.216 -68.6851,121.122 -68.91,119.997 -69.1288,118.858 -69.3389,117.684 -69.5509,116.5 -69.7445,115.285 -69.9301,114.07 -70.1125,112.807 -70.2815,111.558 -70.4429,110.265 -70.5994,108.929 -70.7433,107.607 -70.876,106.277 -70.9954,105.739 -71.

In [3]:
file_titles = []
zip_files = []

for product in products:
    for product_id in product:
        file_titles.append(product[product_id]['title'])
        zip_files.append(product[product_id]['path'])

        import zipfile
        archive = zipfile.ZipFile(product[product_id]['path'], 'r')
        archive.extractall(path=DOWNLOADED_DATA_PATH)

for zip_file in zip_files:
    if REMOVE_ZIPS:
        os.remove(zip_file)    

In [4]:
from netCDF4 import Dataset
for file_title in file_titles:
    data_path = os.path.join(DOWNLOADED_DATA_PATH, '{}.SEN3'.format(file_title))
    for nc_file_name in sorted(os.listdir(data_path)):
        if nc_file_name.endswith('.nc'):
            nc_file_path =  os.path.join(data_path, nc_file_name)  
            print(nc_file_path)
            rootgrp = Dataset(nc_file_path, "r", format="NETCDF4")

            for i in rootgrp.variables:
                print(i, rootgrp.variables[i].shape)

            print('\n')

./src/data/S3A_SL_2_LST____20200531T153925_20200531T154225_20200531T165411_0180_059_025_4680_LN2_O_NR_004.SEN3/LST_ancillary_ds.nc
NDVI (1200, 1500)
NDVI_orphan (1200, 187)
TCWV (1200, 1500)
TCWV_orphan (1200, 187)
biome (1200, 1500)
biome_orphan (1200, 187)
fraction (1200, 1500)
fraction_orphan (1200, 187)
validation (256,)


./src/data/S3A_SL_2_LST____20200531T153925_20200531T154225_20200531T165411_0180_059_025_4680_LN2_O_NR_004.SEN3/LST_in.nc
LST (1200, 1500)
LST_orphan (1200, 187)
LST_uncertainty (1200, 1500)
LST_uncertainty_orphan (1200, 187)
exception (1200, 1500)
exception_orphan (1200, 187)


./src/data/S3A_SL_2_LST____20200531T153925_20200531T154225_20200531T165411_0180_059_025_4680_LN2_O_NR_004.SEN3/cartesian_in.nc
x_in (1200, 1500)
x_orphan_in (1200, 187)
y_in (1200, 1500)
y_orphan_in (1200, 187)


./src/data/S3A_SL_2_LST____20200531T153925_20200531T154225_20200531T165411_0180_059_025_4680_LN2_O_NR_004.SEN3/cartesian_tx.nc
x_tx (1200, 130)
y_tx (1200, 130)


./src/data/S3A_S